In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,32732
2,Huelva,Confirmados PDIA 14 días,1316
3,Huelva,Tasa PDIA 14 días,"256,44523257400084"
4,Huelva,Confirmados PDIA 7 días,549
5,Huelva,Total Confirmados,32914
6,Huelva,Curados,30672
7,Huelva,Fallecidos,388


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  32732.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8927.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 604 personas en los últimos 7 días 

Un positivo PCR cada 258 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,32732.0,549.0,1316.0,513170.0,106.982092,256.445233,102.0
Huelva-Costa,18865.0,325.0,770.0,289548.0,112.243911,265.931728,61.0
Huelva (capital),8927.0,238.0,558.0,143837.0,165.465075,387.939126,44.0
Condado-Campiña,10620.0,174.0,418.0,156231.0,111.373543,267.552534,30.0
Sierra de Huelva-Andévalo Central,2896.0,49.0,124.0,67391.0,72.710006,184.000831,11.0
Valverde del Camino,645.0,26.0,66.0,12750.0,203.921569,517.647059,10.0
Almonte,1888.0,29.0,69.0,24507.0,118.333537,281.552210,10.0
Lepe,2157.0,11.0,29.0,27880.0,39.454806,104.017217,6.0
Cartaya,1369.0,12.0,26.0,20083.0,59.752029,129.462730,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cortelazor,13.0,1.0,5.0,299.0,334.448161,1672.240803,0.0
San Juan del Puerto,559.0,26.0,57.0,9411.0,276.272447,605.674211,2.0
Puebla de Guzmán,136.0,10.0,17.0,3092.0,323.415265,549.805951,2.0
Valverde del Camino,645.0,26.0,66.0,12750.0,203.921569,517.647059,10.0
Huelva (capital),8927.0,238.0,558.0,143837.0,165.465075,387.939126,44.0
Almendro (El),39.0,2.0,3.0,840.0,238.095238,357.142857,0.0
Palos de la Frontera,703.0,14.0,40.0,11742.0,119.230114,340.657469,3.0
Calañas,200.0,8.0,9.0,2768.0,289.017341,325.144509,1.0
Moguer,1361.0,28.0,70.0,21867.0,128.046829,320.117071,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villarrasa,180.0,1.0,7.0,2211.0,45.228403,316.598824,0.0,0.142857
Minas de Riotinto,128.0,1.0,6.0,3812.0,26.232949,157.397692,0.0,0.166667
Punta Umbría,841.0,6.0,31.0,15355.0,39.075220,201.888636,1.0,0.193548
Cortelazor,13.0,1.0,5.0,299.0,334.448161,1672.240803,0.0,0.200000
Manzanilla,103.0,1.0,4.0,2118.0,47.214353,188.857413,0.0,0.250000
Aljaraque,1157.0,16.0,57.0,21474.0,74.508708,265.437273,2.0,0.280702
Bonares,291.0,6.0,19.0,6060.0,99.009901,313.531353,0.0,0.315789
Trigueros,329.0,8.0,24.0,7862.0,101.755279,305.265836,2.0,0.333333
Palos de la Frontera,703.0,14.0,40.0,11742.0,119.230114,340.657469,3.0,0.350000
